In [21]:
import os
os.getcwd()

'/home/jovyan/work/adv_dsi_lab_5/notebooks'

In [22]:
os.chdir('..')

In [23]:
import os
os.getcwd()

'/home/jovyan/work/adv_dsi_lab_5'

# 2.   Load and Explore Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
file_url = 'https://raw.githubusercontent.com/aso-uts/applied_ds/master/unit3/dataset/Car%20Evaluation.csv'

In [4]:
df = pd.read_csv(file_url)

In [5]:
df.head()

,buying_price,maintenance_cost,doors,persons_capacity,luggage_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [6]:
df.shape

(1728, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying_price        1728 non-null object
maintenance_cost    1728 non-null object
doors               1728 non-null object
persons_capacity    1728 non-null object
luggage_boot        1728 non-null object
safety              1728 non-null object
evaluation          1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


In [8]:
df.describe()

,buying_price,maintenance_cost,doors,persons_capacity,luggage_boot,safety,evaluation
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,med,med,4,more,big,med,unacc
freq,432,432,432,576,576,576,1210


In [9]:
df.to_csv('../data/raw/car_evaluation.csv', index=False)

In [10]:
df_cleaned = df.copy()

Create a dictionary called cats_dict that contains the categorical variables as keys and their respective values sorted in ascending order

In [11]:
cats_dict = {
    'buying_price': [['low', 'med', 'high', 'vhigh']],
    'maintenance_cost': [['low', 'med', 'high', 'vhigh']],
    'doors': [['2', '3', '4', '5more']],
    'persons_capacity': [['2', '4', 'more']],
    'luggage_boot': [['small', 'med', 'big']],
    'safety': [['low', 'med', 'high']],
    'evaluation': [['unacc', 'acc', 'good', 'vgood']],
}

In [12]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

Iterate through the elements of cast_dict, instantiate an OrdinalEncoder() and transform the values of each column with this encoder

In [13]:
for col, cats in cats_dict.items():
    col_encoder = OrdinalEncoder(categories=cats)
    df_cleaned[col] = col_encoder.fit_transform(df_cleaned[[col]])

In [14]:
num_cols = ['buying_price', 'maintenance_cost', 'doors', 'persons_capacity', 'luggage_boot', 'safety']

In [16]:
sc = StandardScaler()

Fit and transform the numeric feature of X_train_cleaned and replace the data into it

In [17]:
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

Convert the column evaluation as integer

In [18]:
df_cleaned['evaluation'] = df_cleaned['evaluation'].astype(int)

In [32]:
df_cleaned

,buying_price,maintenance_cost,doors,persons_capacity,luggage_boot,safety,evaluation
0,1.341641,1.341641,-1.341641,-1.224745,-1.224745,-1.224745,0
1,1.341641,1.341641,-1.341641,-1.224745,-1.224745,0.000000,0
2,1.341641,1.341641,-1.341641,-1.224745,-1.224745,1.224745,0
3,1.341641,1.341641,-1.341641,-1.224745,0.000000,-1.224745,0
4,1.341641,1.341641,-1.341641,-1.224745,0.000000,0.000000,0
...,...,...,...,...,...,...,...
1723,-1.341641,-1.341641,1.341641,1.224745,0.000000,0.000000,2
1724,-1.341641,-1.341641,1.341641,1.224745,0.000000,1.224745,3
1725,-1.341641,-1.341641,1.341641,1.224745,1.224745,-1.224745,0
1726,-1.341641,-1.341641,1.341641,1.224745,1.224745,0.000000,2


In [49]:
target_col = 'evaluation'

In [50]:
X = df_cleaned

In [51]:
from adv_dsi_lab_5.src.data.sets import split_sets_by_time, save_sets, split_sets_random

In [52]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col=target_col, test_ratio=0.2, to_numpy=True)

Save the sets in the data/processed/beijing_pollution folder

In [53]:
save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='data/processed/car_evaluation/')

Import this class from src/models/pytorch and convert all sets to PytorchDataset

In [55]:
from adv_dsi_lab_5.src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

# 4. Baseline Model

In [57]:
from adv_dsi_lab_5.src.models.null import NullModel

Instantiate a NullModel and call .fit_predict() on the training target to extract your predictions into a variable called y_base

In [58]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [60]:
from adv_dsi_lab_5.src.models.performance import print_class_perf

In [61]:
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.6988416988416989
F1 Training: 0.5749561249561249


# 5. Define Architecture

Create in src/models/pytorch.py a class called PytorchMultiClass that inherits from nn.Module with:

num_features as input parameter
attributes:
layer_1: fully-connected layer with 32 neurons
layer_out: fully-connected layer with 4 neurons
softmax: softmax function
methods:
forward() with inputs as input parameter, perform ReLU and DropOut on the fully-connected layer followed by the output layer with softmax

class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 32)
        self.layer_out = nn.Linear(32, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)

In [68]:
from adv_dsi_lab_5.src.models.pytorch import PytorchMultiClass

model = PytorchMultiClass(X_train.shape[1])

Set model to use the device available

In [69]:
from adv_dsi_lab_5.src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [70]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


# 6. Train Model

Instantiate a nn.CrossEntropyLoss() and save it into a variable called criterion

In [73]:
criterion = nn.CrossEntropyLoss()

Instantiate a torch.optim.Adam() optimizer with the model's parameters and 0.1 as learning rate and save it into a variable called optimizer

In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [76]:
from adv_dsi_lab_5.src.models.train_classification_model import train_classification
from adv_dsi_lab_5.src.models.test_classification_model import test_classification

In [77]:
N_EPOCHS = 50
BATCH_SIZE = 32

Create a for loop that will iterate through the specified number of epochs and will train the model with the training set and assess the performance on the validation set and print their scores

In [78]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0316	|	Acc: 75.2%
Epoch: 1
	(train)	|	Loss: 0.0299	|	Acc: 80.4%
Epoch: 2
	(train)	|	Loss: 0.0303	|	Acc: 79.2%
Epoch: 3
	(train)	|	Loss: 0.0299	|	Acc: 80.5%
Epoch: 4
	(train)	|	Loss: 0.0294	|	Acc: 82.7%
Epoch: 5
	(train)	|	Loss: 0.0291	|	Acc: 82.4%
Epoch: 6
	(train)	|	Loss: 0.0291	|	Acc: 82.5%
Epoch: 7
	(train)	|	Loss: 0.0289	|	Acc: 83.4%
Epoch: 8
	(train)	|	Loss: 0.0287	|	Acc: 83.9%
Epoch: 9
	(train)	|	Loss: 0.0293	|	Acc: 82.8%
Epoch: 10
	(train)	|	Loss: 0.0287	|	Acc: 84.2%
Epoch: 11
	(train)	|	Loss: 0.0284	|	Acc: 85.4%
Epoch: 12
	(train)	|	Loss: 0.0281	|	Acc: 86.0%
Epoch: 13
	(train)	|	Loss: 0.0285	|	Acc: 84.6%
Epoch: 14
	(train)	|	Loss: 0.0284	|	Acc: 84.7%
Epoch: 15
	(train)	|	Loss: 0.0288	|	Acc: 84.0%
Epoch: 16
	(train)	|	Loss: 0.0280	|	Acc: 86.5%
Epoch: 17
	(train)	|	Loss: 0.0281	|	Acc: 86.1%
Epoch: 18
	(train)	|	Loss: 0.0281	|	Acc: 86.0%
Epoch: 19
	(train)	|	Loss: 0.0286	|	Acc: 84.4%
Epoch: 20
	(train)	|	Loss: 0.0285	|	Acc: 84.8%
Epoch: 21
	(train)	|	Lo

Save model

In [79]:
torch.save(model, "models/pytorch_multi_car_evaluation.pt")

Assess the model performance on the testing set and print its scores

In [80]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0289	|	Accuracy: 0.8
